In [1]:
# 数值特征
import pandas as pd
import numpy as np
import json
from tqdm import tqdm as tqdm
from sklearn.model_selection import KFold

In [2]:
train_data = []
train_data_dict = {}
with open('data/raw_data/train.json', 'r') as f:
    for line in f:
        raw = eval(str(json.loads(line)).lower())
        train_data.append(raw)
        #train_data_dict[raw['text_id']] = raw
kb = {}
with open('data/raw_data/kb_data', 'r') as f:
    for line in f:
        item = eval(str(json.loads(line)).lower())
        kb[item['subject_id']] = item
            
name_id = {}
for kb_id in kb:
    for item in kb[kb_id]['alias']:
        if item not in name_id:
            name_id[item] = [kb_id]
        else:
            name_id[item].append(kb_id)
    if kb[kb_id]['subject'] not in name_id:
        name_id[kb[kb_id]['subject']] = [kb_id]
    else:
        name_id[kb[kb_id]['subject']].append(kb_id)
for id in name_id:
    name_id[id] = sorted(list(set(name_id[id])))

In [3]:
data = pd.read_pickle('data/step1.pkl')
test_data = pd.read_pickle('data/step1_test.pkl')

In [4]:
test_data['m_id']

0         257734
1         257735
2         257735
3         257735
4         257735
5         257735
6         257735
7         257735
8         257735
9         257735
10        257736
11        257736
12        257736
13        257736
14        257736
15        257736
16        257736
17        257736
18        257737
19        257737
20        257737
21        257737
22        257737
23        257737
24        257737
25        257737
26        257737
27        257737
28        257737
29        257737
           ...  
764187    353507
764188    353507
764189    353507
764190    353507
764191    353507
764192    353507
764193    353507
764194    353508
764195    353508
764196    353508
764197    353508
764198    353508
764199    353508
764200    353508
764201    353508
764202    353508
764203    353508
764204    353508
764205    353508
764206    353508
764207    353508
764208    353508
764209    353508
764210    353508
764211    353508
764212    353508
764213    353508
764214    3535

In [4]:
train_match = np.load('model_type/deep_match.npy')

In [5]:
train_match[:,0]

array([9.1322297e-01, 9.6259850e-01, 9.9584371e-01, ..., 8.1027893e-04,
       1.5532824e-02, 9.5632831e-03], dtype=float32)

In [6]:
data['deep_match'] = train_match[:,0]

In [7]:
test_match = np.load('model_type/deep_match_test.npy')

In [8]:
test_match.shape

(3821085, 1)

In [9]:
count = int(test_match.shape[0]/5)

In [10]:
mention_vector = []
for i in range(5):
    mention_vector.append(test_match[i*count:(i+1)*count,:])

In [11]:
mention_vector = sum(mention_vector)/5

In [12]:
mention_vector.shape

(764217, 1)

In [13]:
test_data['deep_match'] = mention_vector

In [14]:
data.columns

Index(['text_id', 'kb_id', 'train_mention', 'label', 'num_attrs',
       'num_abstract_words', 'num_alias', 'm_id', 'num_candidates',
       'mention_equal_subject', 'shuminghao', 'entity_common', 'len_mention',
       'mention_start', 'create_works', 'mention_end', 'end_with_point',
       'end_with_line', 'end_with_line2', 'end_with_shuminghao', 'works_match',
       'numerical_0', 'numerical_1', 'numerical_2', 'numerical_3',
       'numerical_4', 'numerical_5', 'numerical_6', 'numerical_7',
       'numerical_8', 'numerical_9', 'numerical_10', 'numerical_11',
       'numerical_12', 'numerical_13', 'numerical_14', 'type',
       'type_label_mean', 'type_label_count', 'deep_match'],
      dtype='object')

In [15]:
test_data['label']= -1

In [16]:
train_size = data.shape[0]

In [17]:
data= data.append(test_data)

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [22]:
to_rank = ['deep_match']
data_all = data

In [23]:
for f in to_rank:
    data_group = data_all.groupby('m_id')[f].rank(ascending=False)
    data_all['m_rank_'+f] = data_group

In [24]:
for f in tqdm(to_rank):
    data_group = data_all.groupby('m_id')[f].agg({'mid_'+f+'_var':'var','mid_'+f+'_mean':'mean',
                                                  'mid_'+f+'_max':'max','mid_'+f+'_min':'min'})
    data_all = data_all.merge(data_group,on='m_id',how='left')

  0%|          | 0/1 [00:00<?, ?it/s]/home/zhukaihua/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 1/1 [00:02<00:00,  2.24s/it]


In [25]:
for item in data_all.columns:
    if data_all.dtypes[item]=='float64':
        print(item)
        data_all[item] = data_all[item].astype('float32')
    if data_all.dtypes[item]=='int64':
        data_all[item] = data_all[item].astype('int32')
    if data_all.dtypes[item]=='bool':
        data_all[item] = data_all[item].astype('int8')

numerical_0
numerical_1
numerical_10
numerical_11
numerical_12
numerical_13
numerical_14
numerical_2
numerical_3
numerical_4
numerical_5
numerical_6
numerical_7
numerical_8
numerical_9
type_label_count
type_label_mean
m_rank_deep_match


In [26]:
data_all.head()

,create_works,deep_match,end_with_line,end_with_line2,end_with_point,end_with_shuminghao,entity_common,kb_id,label,len_mention,...,train_mention,type,type_label_count,type_label_mean,works_match,m_rank_deep_match,mid_deep_match_var,mid_deep_match_mean,mid_deep_match_max,mid_deep_match_min
0,0,0.913223,0,0,0,0,4,130287,0,4,...,南京南站,place,27241.0,0.336478,1,2.0,0.001219,0.937911,0.962599,0.913223
1,0,0.962599,0,0,0,0,4,311223,1,4,...,南京南站,place,27241.0,0.336478,1,1.0,0.001219,0.937911,0.962599,0.913223
2,0,0.995844,0,0,0,0,1,341096,1,2,...,高铁,thing,499598.0,0.072104,1,1.0,NaN,0.995844,0.995844,0.995844
3,0,0.916084,0,0,0,0,4,130287,0,4,...,南京南站,place,27241.0,0.336478,1,2.0,0.001144,0.940003,0.963922,0.916084
4,0,0.963922,0,0,0,0,4,311223,1,4,...,南京南站,place,27241.0,0.336478,1,1.0,0.001144,0.940003,0.963922,0.916084


In [27]:
data = data_all.iloc[0:train_size,:]
test_data = data_all.iloc[train_size:,:]
data.reset_index(drop=True,inplace=True)
test_data.reset_index(drop=True,inplace=True)

In [28]:
data.to_pickle('data/step4.pkl')
test_data.to_pickle('data/step4_test.pkl')